In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'Summer_2023_Research/polyact_code'

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

import cvxpy as cp
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
import polyact_class as polyact

%load_ext autoreload
%autoreload 2

Mounted at /content/drive


In [4]:
!pip install ucimlrepo

In [5]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_original.data.features
y = breast_cancer_wisconsin_original.data.targets
X = X.fillna(0)

train_frac = 0.8
x_np = X.to_numpy()
y_np = 2 * ((y.to_numpy().squeeze() - 2) / 2) - 1

n,d = x_np.shape
xtrain, xtest = x_np[:int(train_frac*n), :], x_np[int(train_frac*n):,:]
ytrain, ytest = y_np[:int(train_frac*n)], y_np[int(train_frac*n):]
mean, std = xtrain.mean(axis=0, keepdims = True), xtrain.std(axis=0, keepdims=True)
xtrain = (xtrain - mean)/(std + 1e-9)
xtest = (xtest - mean)/(std + 1e-9)

In [7]:
def compute_fgsm(Q, g, x, y, eps, ord = 2, intercept=False):
  grad = -y*(2*(0.09*Q @ x.T)+0.5*g)
  attack = np.sign(grad).T
  if intercept:
    attack[0][-1] = 0
  return eps*attack/np.linalg.norm(attack, ord)

def eval_polyact_fgsm(model, x, y, eps, ord =2):
  Q = (model.weights[(0, 1, 0)] - model.weights[(0, 1, 1)]).value
  g = (model.weights[(0, 2, 0)] - model.weights[(0, 2, 1)]).value
  correct = 0
  n = x.shape[0]
  for i in range(n):
    cur_x, cur_y = x[i:i+1], y[i]
    if np.sign(model.forward(cur_x))[0] == cur_y:
      attack = compute_fgsm(Q, g, cur_x, cur_y, eps, ord=ord)
      correct += model.accuracy(cur_x + attack, cur_y)
  return correct/n

In [8]:
robust_sizes = np.linspace(0.5,2.25,15)
avg_dists = []
clean_accs = []
robust_l2_2dot5 = []
for r in robust_sizes:
  print(r)
  cur_P = polyact.PolyActNN(xtrain, ytrain, r, 0.01, which_class = 1, always_feas = True)
  clean_accs.append(cur_P.accuracy(xtest, ytest))
  robust_l2_2dot5.append(eval_polyact_fgsm(cur_P, xtest, ytest,2.5, ord=2))
  count_robust = 0
  count_nonrobust = 0
  dists_robust = []

  for i, y in enumerate(ytest):
    curx = xtest[i:i+1]
    out_r = np.sign(cur_P.forward(curx))
    if np.sign(out_r) == y:
      count_robust += 1
      dist, gam = cur_P.flip_distance(curx, y)
      dists_robust.append(dist)

  avg_dists.append(sum(dists_robust)/count_robust)

0.5


/content/drive/My Drive/Summer_2023_Research/organized_code/polyact_helpers.py:83: RuntimeWarning: invalid value encountered in sqrt
  alpha = (-2*p_1Gp_j + np.sqrt(discriminant)) / (2*p_jGp_j)


0.625
0.75
0.875
1.0
1.125
1.25
1.375
1.5
1.625
1.75
1.875
2.0
2.125
2.25


In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(robust_sizes[5:], avg_dists[5:],color='red')
ax2.plot(robust_sizes[5:], clean_accs[5:], color='blue', label = r'Clean')
ax2.plot(robust_sizes[5:], robust_l2_2dot5[5:], '--', color='blue', label=r'Robust')

ax1.set_xlabel(r'$r$')
ax1.set_ylabel(r'Average $d_\mathcal{D}(x)$', color= 'red')
ax1.tick_params(axis="y", labelcolor = 'red')

ax2.set_ylabel("Test accuracy", color = 'blue')
ax2.tick_params(axis="y", labelcolor = 'blue')

fig.autofmt_xdate()
ax2.legend()
plt.savefig('REPRODUCED_accs_and_dists_vs_r.png')